# CrewAI Markdown Validator


First let's install the required packages

In [3]:
%pip install -U crewai
%pip install -U agentops
%pip install -U python-dotenv
%pip install -U langchain_openai
%pip install -U langchain_groq
%pip install -U langchain
%pip install -U StringIO
%pip install -U pymarkdown

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement StringIO (from versions: none)
ERROR: No matching distribution found for StringIO
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Then import them

In [4]:
import sys
from crewai import Agent, Task
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain_openai import ChatOpenAI
import agentops
import os
from dotenv import load_dotenv
from pymarkdown.api import PyMarkdownApi, PyMarkdownApiException
from io import StringIO

ModuleNotFoundError: No module named 'StringIO'

Next, we'll grab our API keys. You can use dotenv like below or however else you like to load environment variables

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<your_openai_key>"
GROQ_API_KEY = os.getenv("GROQ_API_KEY") or "<your_groq_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

The first step in any AgentOps integration is to call `agentops.init()`

In [ ]:
agentops.init(AGENTOPS_API_KEY, default_tags=["markdown_validator"])

Lets start by creating our markdown validator tool

In [ ]:
@tool("markdown_validation_tool")
def markdown_validation_tool(file_path: str) -> str:
    """
    A tool to review files for markdown syntax errors.

    Returns:
    - validation_results: A list of validation results
    and suggestions on how to fix them.
    """

    print("\n\nValidating Markdown syntax...\n\n" + file_path)

    try:
        if not (os.path.exists(file_path)):
            return "Could not validate file. The provided file path does not exist."

        scan_result = PyMarkdownApi().scan_path(file_path.rstrip().lstrip())
        results = str(scan_result)
        return results  # Return the reviewed document
    except PyMarkdownApiException as this_exception:
        print(f"API Exception: {this_exception}", file=sys.stderr)
        return f"API Exception: {str(this_exception)}"

In [ ]:
groq_llm = ChatGroq(
    temperature=0,
    groq_api_key=GROQ_API_KEY,
    model_name="llama3-70b-8192",
)

default_llm = ChatOpenAI(
    openai_api_base=os.environ.get("OPENAI_API_BASE_URL", "https://api.openai.com/v1"),
    openai_api_key=OPENAI_API_KEY,
    temperature=0.1,
    model_name=os.environ.get("MODEL_NAME", "gpt-3.5-turbo"),
)

In [ ]:
filename = "README.md"

Lets create our Agent with CrewAI

In [ ]:
general_agent = Agent(
    role="Requirements Manager",
    goal="""Provide a detailed list of the markdown 
                            linting results. Give a summary with actionable 
                            tasks to address the validation results. Write your 
                            response as if you were handing it to a developer 
                            to fix the issues.
                            DO NOT provide examples of how to fix the issues or
                            recommend other tools to use.""",
    backstory="""You are an expert business analyst 
					and software QA specialist. You provide high quality, 
                    thorough, insightful and actionable feedback via 
                    detailed list of changes and actionable tasks.""",
    allow_delegation=False,
    verbose=True,
    tools=[markdown_validation_tool],
    llm=default_llm,
)  # groq_llm)

Now lets create the task for our agent to complete

In [ ]:
syntax_review_task = Task(
    description=f"""
        Use the markdown_validation_tool to review 
        the file(s) at this path: {filename}
        
        Be sure to pass only the file path to the markdown_validation_tool.
        Use the following format to call the markdown_validation_tool:
        Do I need to use a tool? Yes
        Action: markdown_validation_tool
        Action Input: {filename}

        Get the validation results from the tool 
        and then summarize it into a list of changes
        the developer should make to the document.
        DO NOT recommend ways to update the document.
        DO NOT change any of the content of the document or
        add content to it. It is critical to your task to
        only respond with a list of changes.
        
        If you already know the answer or if you do not need 
        to use a tool, return it as your Final Answer.""",
    agent=general_agent,
    expected_output="",
)

Now lets run our task!

In [ ]:
syntax_review_task.execute_sync()

Finally, don't forget to end your AgentOps session!

In [ ]:
agentops.end_session("Success")